In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
import quadruppedEnv
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
env = env_configurations.a2c_configurations[env_name]['ENV_CREATOR']()
print(env.action_space.low)
print(env.action_space.high)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-05-23 21:59:02,916	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-23_21-59-02_5466/logs.
2019-05-23 21:59:03,029	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:27664 to respond...
2019-05-23 21:59:03,147	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:29918 to respond...
2019-05-23 21:59:03,151	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-05-23 21:59:03,193	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-23_21-59-02_5466/logs.
2019-05-23 21:59:03,196	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.104',
 'redis_address': '192.168.3.104:27664',
 'object_store_address': '/tmp/ray/session_2019-05-23_21-59-02_5466/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-23_21-59-02_5466/sockets/raylet',
 'webui_url': None}

In [5]:
from a2c_v2 import A2CAgent
import tr_helpers
import networks
import a2c_games_configurations

a2c_config = a2c_games_configurations.quadrupped_config
#a2c_config = a2c_games_configurations.roboschoolant_config
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
action_space = env.action_space

agent = A2CAgent(sess,'rob_20', env.observation_space, False, action_space, a2c_config)
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')
#agent.train()


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


2019-05-23 21:59:04,490	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


In [6]:
ray.shutdown()

In [7]:
import numpy as np
#agent.save('nn/latest')
agent.restore('nn/rob_20QuadruppedWalk-v1')
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')

def evaluate(env,t_max=5000):
    rewards = []
    env._max_episode_steps = 5000
    env.batch_mode = False
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env_monitor.close()




Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from nn/rob_20QuadruppedWalk-v1
reset
[[ 1.         -0.3429913  -0.09194368 -0.38524985  0.0526244  -0.8895143
   0.26580852 -0.00988311 -0.25902343 -0.1842013   0.61536205 -0.41956505]]
[[ 0.49605718 -0.21468249  0.91201884 -0.7026969  -0.503405   -0.53142
   0.6137022   0.2974256   0.5925346  -0.666262    0.8331404  -0.83763003]]
[[ 3.20322961e-01 -3.44978452e-01  1.09577507e-01 -1.06713057e-01
  -4.35576499e-01  1.43194124e-01  3.62104684e-01  1.79763883e-04
   9.12854224e-02 -5.91892242e-01  1.20789364e-01  2.62170404e-01]]
[[ 0.68098474 -0.27960888  0.24495828 -0.70799434 -0.25675336 -0.88123184
   0.7730254   0.62005377  0.3460164  -0.64092577 -0.06905653  0.21371083]]
[[ 0.38718313  0.29923072  0.3912623  -0.59980845 -0.6911799  -0.05627544
   0.7842294  -0.44860196  0.655295   -0.24615115  0.53709126  0.26042768]]
[[ 0.5084483  -0.61038077 -0.09359178 -0.5

[[ 0.6436916  -0.5209078   0.22571062 -0.23854935  0.27980733 -0.6432406
   0.7405167  -0.18667969  0.5911384  -0.7812623  -0.3311623   0.40332907]]
[[ 0.598514   -0.07890543  0.48342294 -0.5116106  -0.81098384 -0.712056
   0.781093    1.         -0.02722993 -0.3695653   0.2473222   0.22955666]]
[[ 0.8098961  -0.3885367   0.4844013  -0.71888125 -0.05149037 -0.23666216
   0.7315251   0.23388627  0.6959582  -0.6723948   0.44511753  0.27902317]]
[[ 0.67117304 -0.38109294  0.47237447 -0.47405845  0.24303529  0.08228575
   0.76357687 -0.37283134  0.3653934  -0.81304294 -0.46294042  0.3994974 ]]
[[ 0.64258695  0.07191396  0.2211615  -0.38966593  0.01504248 -0.17190546
   0.90343666 -0.18279111  0.7713047  -0.51410705 -0.306027   -0.05732906]]
[[ 0.5553759   0.05781545  0.5789077  -0.49822983 -0.45111352 -0.4810412
   0.74005574  0.18425737  0.34534776 -0.22538495  0.42924845  0.4165302 ]]
[[ 0.94226474 -0.31229457  0.4909877  -0.17077884 -0.55054766 -0.24954808
   0.7270602  -0.42952782  0.4

[[ 0.47818285 -0.64907295  0.20455112 -0.72758067  0.3259934   0.67917347
   0.8068954  -0.29590353 -0.19616172 -0.5239617   0.36888397  0.50032616]]
[[ 0.7854957  -0.60444856  0.43172103 -0.61356467  0.24948019 -0.00948276
   1.          0.23349464  1.         -0.4333708   0.46384045  0.45233637]]
[[ 0.68231833 -0.7216736   0.09738688 -0.44081882  0.38081622 -0.05986415
   0.8100396  -0.00341174  0.13564894 -0.81321764 -0.3588895   0.6496364 ]]
[[ 0.65471125 -0.17118584  0.2614333  -0.5968876   0.14847068 -0.43264776
   0.71072763  0.62939286  0.43694028 -0.5648367  -0.4243936   0.2819436 ]]
[[ 0.38552815 -0.16377029  0.27515602 -0.5656579  -0.17043152 -0.26061895
   0.4603628   0.11237097  0.21173772 -0.69631964 -0.18189403  0.2587138 ]]
[[ 2.1399975e-01  4.3124458e-01  6.0964310e-01 -5.8001947e-01
  -9.5692772e-01  2.1644202e-01  8.2206488e-01  5.0686061e-02
   7.9652178e-01 -8.8959056e-01  2.2567460e-01 -8.4628165e-04]]
[[ 0.45780534 -0.25242007 -0.1960021  -0.8334803  -0.44523963 

[[ 0.01589161  0.30800515  0.86939406 -0.79066133 -0.7518142  -0.4765411
   0.76135707  0.25859377 -0.01453985 -0.05212522  0.52571005  0.6266396 ]]
[[ 0.50645375 -0.21348915 -0.21869743 -0.6222656  -0.46037355  0.7252163
   0.81711984 -0.39852983  0.3217498  -0.78097767  0.48767662  0.1397554 ]]
[[ 0.44457734 -0.6785664   0.60024214 -0.6638407   0.38793045  0.88975406
   0.52166593 -0.1635133   0.2439478  -0.8995457   0.16316575  0.84713334]]
[[ 0.6720986  -0.7836033   0.4993956  -0.75879925  0.5606892  -0.09641923
   0.97135425  0.14285901  1.         -0.94651717 -0.11489686  0.57503676]]
[[ 0.8581477  -0.15639248  0.4898619  -0.6777518   0.01166841 -0.6209108
   0.94956183  0.0687253   0.64258915 -0.40078703 -0.24410515  0.4012655 ]]
[[ 0.5557889   0.42217442  1.         -0.823292   -0.8561072  -0.7085379
   0.8925071   0.3934409   0.10500297 -0.5212609   0.1902052  -0.22072381]]
[[ 0.62612057 -0.06649369  0.09557918 -0.9995315  -0.76793206  0.26026103
   0.5950913  -0.5164083  -0.0

[[ 0.6084949   0.28030488  0.89377403 -0.45616937 -0.44673073 -0.10296151
   0.7154831   0.16865493  1.         -0.668235   -0.0398637   0.3211914 ]]
[[ 0.34014487  0.27727795  0.48090687 -0.6956131  -0.9039825  -0.12976474
   0.5749152  -0.38357326 -0.33385012 -0.62216264  0.638915   -0.01533714]]
[[ 0.514674   -0.1525722   0.45040554 -0.5906218  -0.41301537  0.34583384
   0.51831275 -0.10097745  0.04718246 -0.33057284  0.5908638   0.24835855]]
[[ 0.54380393 -0.57895625  0.5024241  -0.4366529   0.08493054  0.49691907
   0.54907227 -0.15982223  0.69167876 -0.59600055  0.3259219   0.280218  ]]
[[ 0.661058   -0.4741686   0.27018574 -0.22596116  0.5529239  -0.56810886
   0.9678799   0.46351376  0.56489265 -0.5552904  -0.5607456   0.6900488 ]]
[[ 0.76729476 -0.7699677   0.32626265 -0.30890083  0.45034206 -0.2283257
   0.5229646   0.06229566  0.7025147  -0.42564368 -0.29387826  0.48546767]]
[[ 0.40187198  0.11534972  0.61791867 -0.47315258 -0.27285784 -0.31655458
   0.5260098   0.33824003  

KeyboardInterrupt: 

In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()